In [1]:
import keras
from keras.models import Model
from keras.layers import Conv1D,Dense,Flatten,Concatenate,BatchNormalization,Input,Embedding
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import tensorflowjs as tfjs

import numpy as np
import pandas as pd

Using TensorFlow backend.


In [10]:
TRAIN_FILE = '../train.csv'
TEST_FILE = '../test.csv'
TOKENIZER = Tokenizer(num_words=10000)
                     

In [28]:
def datify(text_file):
    sentences=[]
    sentiments=[]
    maxlength =0
    with open(text_file,errors='ignore') as f:
        f.readline()
        for line in f:
            try:
                _,sentiment,sentence = line.split(',')
                sentences.append(sentence)
                len_sent = len(sentence)
                if(maxlength<len_sent):
                    maxlength = len_sent
                sentiments.append(sentiment)
            except:
                pass
    TOKENIZER.fit_on_texts(sentences)
    sentences = TOKENIZER.texts_to_sequences(sentences)
    sentences = pad_sequences(sentences,maxlen = maxlength)
    return sentences,sentiments,maxlength
    

In [29]:
train_x,train_y,train_length = datify(TRAIN_FILE)

In [42]:
# layer definition
vocab_size = len(TOKENIZER.word_index)+1
input1 = Input(shape=(train_length,))
embedding1 = Embedding(vocab_size,100)(input1)
layer1_group1_conv = Conv1D(filters=32,kernel_size=2,activation='relu')(embedding1)
layer1_group1_bn = BatchNormalization()(layer1_group1_conv)
layer1_group1_flat = Flatten()(layer1_group1_bn)
input2 = Input(shape=(train_length,))
embedding2 = Embedding(vocab_size,100)(input2)
layer1_group2_conv = Conv1D(filters=32,kernel_size=3,activation='relu')(embedding2)
layer1_group2_bn = BatchNormalization()(layer1_group2_conv)
layer1_group2_flat = Flatten()(layer1_group2_bn)
input3 = Input(shape=(train_length,))
embedding3 = Embedding(vocab_size,100)(input3)
layer1_group3_conv = Conv1D(filters=32,kernel_size=4,activation='relu')(embedding3)
layer1_group3_bn = BatchNormalization()(layer1_group3_conv)
layer1_group3_flat = Flatten()(layer1_group3_bn)
layer2_concat = Concatenate()([layer1_group1_flat,layer1_group2_flat,layer1_group3_flat])
layer3_dense = Dense(10,activation='relu')(layer2_concat)
layer4_out = Dense(1,activation='sigmoid')(layer3_dense)


In [43]:
#model definition
model = Model(inputs=[input1,input2,input3],outputs=layer4_out)

In [44]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [48]:
from keras.utils import plot_model
plot_model(model,to_file='../images/model.png',show_shapes=True)

In [54]:
train_x = np.array(train_x)
train_y = np.array(train_y)

In [58]:
model.fit([train_x,train_x,train_x],train_y,epochs=3,batch_size=32) #4

Epoch 1/3
75206/75206 [==============================] - 1324s 18ms/step - loss: 0.0882 - acc: 0.9654
Epoch 2/3
75206/75206 [==============================] - 1388s 18ms/step - loss: 0.0679 - acc: 0.9730
Epoch 3/3
75206/75206 [==============================] - 1276s 17ms/step - loss: 0.0603 - acc: 0.9753


In [3]:
from keras.models import load_model
model = load_model('model.h5')
model.save('../model/model.h5')

In [57]:
tfjs.converters.save_keras_model('../model')